# 01A Data Cleansing Weather

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

## Read Data

In [2]:
weather_df = pd.read_csv('../west_nile/west_nile/input/weather.csv')

## One Hot Encode CodeSum Data

In [3]:
codes = {code: [0]*weather_df.shape[0] for codes in weather_df.CodeSum.unique() for code in codes.split()}

weather_df = weather_df.join(pd.DataFrame(codes))
for i, codesum in enumerate(weather_df.CodeSum):
    for code in codesum.split():
        weather_df.at[i, code] = 1
        
codes_df = weather_df.groupby('Date', as_index=True)[list(codes.keys())].max()

## Calculate Missing Tavg Values

In [4]:
missing_Tavg = weather_df[weather_df.Tavg == 'M'].index

weather_df.loc[missing_Tavg, 'Tavg'] = round((weather_df.loc[missing_Tavg, 'Tmax'] 
                                              + weather_df.loc[missing_Tavg, 'Tmin'])/2)

## Convert Tavg to Int

In [5]:
weather_df.Tavg = weather_df.Tavg.astype(int)

In [6]:
temperature_df = weather_df.groupby('Date')['Tavg','Tmax','Tmin'].mean()

## Drop Unnecessary Columns
- Incorporate Adam's Work

In [7]:
cols_drop = ['Depart','DewPoint','WetBulb','Heat','Cool','Sunrise','Sunset','CodeSum','Depth','Water1',
             'SnowFall','PrecipTotal','StnPressure','SeaLevel','ResultSpeed','ResultDir','AvgSpeed']
weather_df = weather_df.drop(labels=cols_drop, axis=1)

## Export codes_df and temperature_df

In [8]:
with open('../Cleansed_Data/codes_df.pkl', 'wb') as f:
    pickle.dump(codes_df, f)
    
with open('../Cleansed_Data/temperature_df.pkl', 'wb') as f:
    pickle.dump(temperature_df, f)

## Try to incorporate some of Adam's stuff

In [19]:
weather = pd.read_csv('../west_nile/west_nile/input/weather.csv')

In [20]:
def celsius(x):
    c = ((x - 32) * 5.0)/9.0
    return float(c)

In [21]:
missing_Tavg = weather[weather.Tavg == 'M'].index

weather.loc[missing_Tavg, 'Tavg'] = round((weather.loc[missing_Tavg, 'Tmax'] 
                                              + weather.loc[missing_Tavg, 'Tmin'])/2)

In [24]:
weather.Tavg = weather.Tavg.astype(int)

In [25]:
weather['TavgC'] = weather['Tavg'].apply(celsius)
weather['TminC'] = weather['Tmin'].apply(celsius)
weather['TmaxC'] = weather['Tmax'].apply(celsius)

In [27]:
weather['relative_humidity'] = 100 * (np.exp((17.625 * weather.DewPoint.apply(celsius)) / (243.04 + weather.DewPoint.apply(celsius))) / np.exp((17.625 * weather.Tavg.apply(celsius)) / (243.04 + weather.Tavg.apply(celsius))))

In [28]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,TavgC,relative_humidity
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,M,0.0,0.00,29.10,29.82,1.7,27,9.2,19.444444,56.438576
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,M,M,0.00,29.18,29.82,2.7,25,9.6,20.000000,54.526426
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,M,0.0,0.00,29.38,30.09,13.0,4,13.4,10.555556,71.195808
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,M,M,0.00,29.44,30.08,13.3,2,13.4,11.111111,68.614962
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,M,0.0,0.00,29.39,30.12,11.7,7,11.9,13.333333,54.874919


In [34]:
weather.count()

Station              2944
Date                 2944
Tmax                 2944
Tmin                 2944
Tavg                 2944
Depart               2944
DewPoint             2944
WetBulb              2944
Heat                 2944
Cool                 2944
Sunrise              2944
Sunset               2944
CodeSum              2944
Depth                2944
Water1               2944
SnowFall             2944
PrecipTotal          2944
StnPressure          2944
SeaLevel             2944
ResultSpeed          2944
ResultDir            2944
AvgSpeed             2944
TavgC                2944
relative_humidity    2944
dtype: int64

In [39]:
weather[['Station','Date','Sunrise', 'Sunset']].head(10)

,Station,Date,Sunrise,Sunset
0,1,2007-05-01,0448,1849
1,2,2007-05-01,-,-
2,1,2007-05-02,0447,1850
3,2,2007-05-02,-,-
4,1,2007-05-03,0446,1851
5,2,2007-05-03,-,-
6,1,2007-05-04,0444,1852
7,2,2007-05-04,-,-
8,1,2007-05-05,0443,1853
9,2,2007-05-05,-,-


In [40]:
1849 - 448

1401

6:49 - 4:48 => 

In [42]:
sun = weather[weather.Station == 1]

In [ ]:
df['LatAndLong'] = df.apply(lambda x: str(x.Latitude) +', ' + str(x.Longitude), axis=1)

In [44]:
sun['sundiff'] = int(sun.Sunset) - int(sun.Sunrise)

TypeError: cannot convert the series to <class 'int'>

In [50]:
sun['sundiff'] = sun.apply(lambda x: int(x.Sunset) - int(x.Sunrise), axis=1)

/Users/yibingchen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [51]:
sun.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,TavgC,relative_humidity,sundiff
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,0.0,0.00,29.10,29.82,1.7,27,9.2,19.444444,56.438576,1401
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,0.0,0.00,29.38,30.09,13.0,4,13.4,10.555556,71.195808,1403
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,0.0,0.00,29.39,30.12,11.7,7,11.9,13.333333,54.874919,1405
6,1,2007-05-04,66,49,58,4,41,50,7,0,...,0.0,T,29.31,30.05,10.4,8,10.8,14.444444,53.075889,1408
8,1,2007-05-05,66,53,60,5,38,49,5,0,...,0.0,T,29.40,30.10,11.7,7,12.0,15.555556,43.965942,1410


In [ ]:
sun['photo_period']

In [52]:
sun.sundiff.dtypes

dtype('int64')

In [64]:
sun['photoperiod'] =sun.sundiff.map(lambda x: int(str(x)[:2]) + int(str(x)[-2:])/60)

/Users/yibingchen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [65]:
sun.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,TavgC,relative_humidity,sundiff,photoperiod
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,0.00,29.10,29.82,1.7,27,9.2,19.444444,56.438576,1401,14.016667
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,0.00,29.38,30.09,13.0,4,13.4,10.555556,71.195808,1403,14.050000
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,0.00,29.39,30.12,11.7,7,11.9,13.333333,54.874919,1405,14.083333
6,1,2007-05-04,66,49,58,4,41,50,7,0,...,T,29.31,30.05,10.4,8,10.8,14.444444,53.075889,1408,14.133333
8,1,2007-05-05,66,53,60,5,38,49,5,0,...,T,29.40,30.10,11.7,7,12.0,15.555556,43.965942,1410,14.166667


In [67]:
weather.columns

Index(['Station', 'Date', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint',
       'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum', 'Depth',
       'Water1', 'SnowFall', 'PrecipTotal', 'StnPressure', 'SeaLevel',
       'ResultSpeed', 'ResultDir', 'AvgSpeed', 'TavgC', 'relative_humidity'],
      dtype='object')

In [71]:
weather.AvgSpeed.dtypes

dtype('O')

In [79]:
test

1401

In [82]:
median_wind = weather[weather.AvgSpeed!='M'].AvgSpeed.median()

8.1

In [83]:
median_wind = weather[weather.AvgSpeed!='M'].AvgSpeed.median()
weather.AvgSpeed.replace('M', median_wind, inplace=True);

In [97]:
weather.AvgSpeed.dtypes

dtype('O')

In [98]:
weather.AvgSpeed = weather.AvgSpeed.astype(float)

- Photoperiod
- wind speed
- relative humidity
- Tavg

In [ ]:
weather['ideal_condition'] = 

In [ ]:
weather.apply(lambda x: x.Tavg)

In [ ]:
def check_ideal(row):
    if row.TavgC >= 10 & row.TavgC <= 27:
        ideal_temp = 1
    else:
        ideal_temp = 0
    
    

In [86]:
weather.Tmax.isnull().sum()

0

In [100]:
weather.loc[(weather['TavgC'] >= 10) & (weather['TavgC'] <= 30) & (weather['relative_humidity'] < 95) & (weather['relative_humidity'] > 40) & (weather['AvgSpeed'] < 6), 'ideal_environment']=1
weather.ideal_environment.fillna(0, inplace=True)

In [101]:
weather.ideal_environment.value_counts()

0.0    2365
1.0     579
Name: ideal_environment, dtype: int64

In [121]:
weather[weather.ideal_environment==0][]

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,TavgC,relative_humidity,ideal_environment
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,0.0,0.00,29.10,29.82,1.7,27,9.2,19.444444,56.438576,0.0
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,M,0.00,29.18,29.82,2.7,25,9.6,20.000000,54.526426,0.0
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,0.0,0.00,29.38,30.09,13.0,4,13.4,10.555556,71.195808,0.0
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,M,0.00,29.44,30.08,13.3,2,13.4,11.111111,68.614962,0.0
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,0.0,0.00,29.39,30.12,11.7,7,11.9,13.333333,54.874919,0.0
5,2,2007-05-03,67,48,58,M,40,50,7,0,...,M,0.00,29.46,30.12,12.9,6,13.2,14.444444,51.057657,0.0
6,1,2007-05-04,66,49,58,4,41,50,7,0,...,0.0,T,29.31,30.05,10.4,8,10.8,14.444444,53.075889,0.0
7,2,2007-05-04,78,51,64,M,42,50,M,M,...,M,0.00,29.36,30.04,10.1,7,10.4,17.777778,44.598489,0.0
8,1,2007-05-05,66,53,60,5,38,49,5,0,...,0.0,T,29.40,30.10,11.7,7,12.0,15.555556,43.965942,0.0
9,2,2007-05-05,66,54,60,M,39,50,5,0,...,M,T,29.46,30.09,11.2,7,11.5,15.555556,45.719851,0.0


In [109]:
nminus_df = sun[['Date','relative_humidity']]

In [111]:
nminus_df.head()

,Date,relative_humidity
0,2007-05-01,56.438576
2,2007-05-02,71.195808
4,2007-05-03,54.874919
6,2007-05-04,53.075889
8,2007-05-05,43.965942


In [113]:
!ls ../Cleansed_Data/nminus_df.pkl

Bing - Archive Folder        Notebooks-WIP
Cleansed_Data                West-Nile-Virus-MASTER.ipynb
Models                       West-Nile-Virus-YH.ipynb
Notebooks-Master             west_nile


In [115]:
with open('../Cleansed_Data/nminus_df.pkl', 'wb') as f:
    pickle.dump(nminus_df, f)

In [117]:
with open('../Cleansed_Data/nminus_df.pkl', 'rb') as f:
    nminus_df = pickle.load(f)